In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Reshape
from keras.layers.recurrent import GRU
from keras.optimizers import Adam
import numpy as np
import random
import sys
import os
import codecs
import collections
from six.moves import cPickle
import math

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def encode(y_train):
    target=[]
    for y in y_train:
        if y=='benign':
            target.append(0)
        else:
            target.append(1)
    target = np.asarray(target)
    return target


In [3]:
unique = []
for y in y_train:
    if y not in unique:
        unique.append(y)
print unique

NameError: name 'y_train' is not defined

In [4]:
def get_data(name):
    df = pd.read_csv(name,delimiter=',')
    arr = df.as_matrix()
    l = len(arr[0])
    x_train = arr[:, :l-1]
    y_train = arr[:,l-1]
    y_train = encode(y_train)
    return x_train, y_train

In [5]:
x_train,y_train = get_data('train.csv')
x_test,y_test = get_data('test.csv')
print x_train.shape

(140, 8)


In [6]:
'''if x_train!=x_test:
    print 1'''

'if x_train!=x_test:\n    print 1'

## SVM

In [7]:
C = 0.5
lin_svc = svm.LinearSVC(C=C).fit(x_train, y_train)

In [8]:
y_pred = lin_svc.predict(x_test)
print "Accuracy is ", accuracy_score(y_test,y_pred)*100
print confusion_matrix(y_test, y_pred)

Accuracy is  61.66666666666667
[[27  3]
 [20 10]]


## Random forest

In [9]:
clf = RandomForestClassifier()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print "Accuracy is ", accuracy_score(y_test,y_pred)*100
print confusion_matrix(y_test, y_pred)

Accuracy is  50.0
[[30  0]
 [30  0]]


## KNN

In [10]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [11]:
y_pred = knn.predict(x_test)
print "Accuracy is ", accuracy_score(y_test,y_pred)*100
print confusion_matrix(y_test, y_pred)

Accuracy is  56.666666666666664
[[19 11]
 [15 15]]


## Logistic Regression

In [12]:
from sklearn.linear_model import LogisticRegression
logisticRegr = LogisticRegression()
logisticRegr.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
y_pred = logisticRegr.predict(x_test)
print "Accuracy is ", accuracy_score(y_test,y_pred)*100
print confusion_matrix(y_test, y_pred)

Accuracy is  53.333333333333336
[[30  0]
 [28  2]]


## Neural Networks

In [14]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print "Accuracy is ", accuracy_score(y_test,y_pred)*100
print confusion_matrix(y_test, y_pred)

Accuracy is  50.0
[[30  0]
 [30  0]]


## RNN

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [16]:
model = Sequential()
model.add(Dense(8, input_dim=8, activation='relu'))
model.add(Dense(1, activation='relu'))
# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train,y_train)

Epoch 1/1


InvalidArgumentError: Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 1 1 1 0 0 0 0 1 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0
	 [[Node: loss/dense_2_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](loss/dense_2_loss/Reshape_1, loss/dense_2_loss/Cast)]]

Caused by op u'loss/dense_2_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_inst